### **PIP**

In [1]:
!pip install -q kaggle

In [2]:
!pip install tensorflow --upgrade
!pip install keras --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 101.4 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras

### ***Import***

In [3]:
import os
import json
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

### **Data collection from kagge**

In [6]:
kaggle_dictionary = json.load(open("/content/kaggle.json"))

In [7]:
#setup kaggle credential as environment variables
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [8]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 62% 16.0M/25.7M [00:00<00:00, 165MB/s]
100% 25.7M/25.7M [00:00<00:00, 198MB/s]


In [9]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [10]:
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
    zip_ref.extractall()

In [11]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


In [12]:
data = pd.read_csv("/content/IMDB Dataset.csv")

In [13]:
data.sentiment.value_counts()

,count
sentiment,
positive,25000
negative,25000


In [14]:
data.replace({"sentiment":{"positive":1, "negative":0}}, inplace = True)

<ipython-input-14-6ae2496545df>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment":{"positive":1, "negative":0}}, inplace = True)


In [15]:
#split the training data
train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 42)

In [16]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


### ***Data Preprocessing***

In [ ]:
# tokenization text dada
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train_seq = tokenizer.texts_to_sequences(train_data["review"])
X_test_seq = tokenizer.texts_to_sequences(test_data["review"])
# Pad the sequences to ensure equal length
X_train = pad_sequences(X_train_seq, maxlen=200)
X_test = pad_sequences(X_test_seq, maxlen=200)

In [ ]:
y_train = train_data["sentiment"]
y_test = test_data["sentiment"]

### **LSTM_long short -term memory**

In [36]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

112281


In [46]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))  # input_shape is implied here
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))
model.build(input_shape=(None, 200))

In [47]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [48]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [49]:
# # Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 183s 348ms/step - accuracy: 0.7314 - loss: 0.5319 - val_accuracy: 0.8226 - val_loss: 0.4115
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 198s 353ms/step - accuracy: 0.8437 - loss: 0.3705 - val_accuracy: 0.8576 - val_loss: 0.3431
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 207s 362ms/step - accuracy: 0.8712 - loss: 0.3162 - val_accuracy: 0.8331 - val_loss: 0.3996
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 184s 326ms/step - accuracy: 0.8782 - loss: 0.3026 - val_accuracy: 0.8503 - val_loss: 0.3699
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 201s 324ms/step - accuracy: 0.8829 - loss: 0.2840 - val_accuracy: 0.8564 - val_loss: 0.3501


In [50]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"test loss: {loss}")
print(f"test accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 32s 101ms/step - accuracy: 0.8657 - loss: 0.3377
test loss: 0.3381136357784271
test accuracy: 0.8654000163078308


In [53]:
def predict_sentiment(review):
    sequence = tokenizer.texts_to_sequences([review])
    pad_seq = pad_sequences(sequence, maxlen=200)
    prediction = model.predict(pad_seq)
    sentiment = "Positive" if prediction[0][0] >0.5 else "Negative"
    return sentiment

In [63]:
new_review = "fantastic"
sentiment = predict_sentiment(new_review)
print(f"the sentiment of movie is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
the sentiment of movie is: Positive


In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer
sentences = ["This is a cat", "This is a dog", "My name is maran winchester"]
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences)
print(tokenizer.word_index)
sequences = tokenizer.texts_to_sequences(sentences)
print(sequences)
padded = pad_sequences(sequences, maxlen=5)
print(padded)

{'is': 1, 'this': 2, 'a': 3, 'cat': 4, 'dog': 5, 'my': 6, 'name': 7, 'maran': 8, 'winchester': 9}
[[2, 1, 3, 4], [2, 1, 3, 5], [6, 7, 1, 8, 9]]
[[0 2 1 3 4]
 [0 2 1 3 5]
 [6 7 1 8 9]]
